In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import keras

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

print(os.listdir("../input"))

In [14]:
train = pd.read_csv('../input/train.csv')
train.head()

In [15]:
test = pd.read_csv('../input/test.csv')
test.head()

In [16]:
train.info()

In [17]:
X = train[['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']].astype('float32')
Y = train['count'].astype('float32')

In [18]:
split = 0.20
seed = 415
X, Y = shuffle(X, Y, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=split, random_state=seed)

In [19]:
train.isnull().sum()

In [20]:
X.shape

In [21]:
print("train_x.shape",train_x.shape)
print("train_y.shape",train_y.shape)
print("test_x.shape",test_x.shape)
print("test_y.shape",test_y.shape)

In [22]:
learning_rate = 0.3
training_epochs = 5
cost_history = np.empty(shape=[1], dtype=float)
Y.shape

In [23]:
_X = tf.placeholder(tf.float32,shape=[None,8])
_Y = tf.placeholder(tf.float32,shape=[None])


sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

In [24]:
# Model architecture parameters
n = 8
n_neurons_1 = 60
n_neurons_2 = 60
n_neurons_3 = 60
n_neurons_4 = 60
n_target = 1
# Layer 1: Variables for hidden weights and biases
W_hidden_1 = tf.Variable(weight_initializer([n, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
# Layer 2: Variables for hidden weights and biases
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
# Layer 3: Variables for hidden weights and biases
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
# Layer 4: Variables for hidden weights and biases
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

# Output layer: Variables for output weights and biases
W_out = tf.Variable(weight_initializer([n_neurons_4, n_target]))
bias_out = tf.Variable(bias_initializer([n_target]))


In [25]:
# Hidden layer
hidden_1 = tf.nn.relu(tf.add(tf.matmul(_X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

# Output layer (must be transposed)
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out))
#out = tf.reshape(out,(train_x.shape[0],1))
hidden_4.shape

In [26]:
# Define the cost function and optimizer
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=_Y))

In [27]:
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [28]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
mse_history = []
accuracy_history = []

In [29]:
#train_y = pd.DataFrame(train_y.values)

In [30]:

#Calculate the cost and the accuracy for each epoch
for epoch in range(training_epochs):
    sess.run(training_step, feed_dict={_X:train_x, _Y:train_y})
    cost = sess.run(cost_function,feed_dict={_X:train_x, _Y:train_y})
    cost_history = np.append(cost_history, cost)
    correct_prediction = tf.equal(tf.argmax(Y,-1),tf.argmax(_Y,-1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    #print("Accuracy: ", (sess.run(accuracy, feed_dict={_X:test_x, _Y:test_y})))
    pred_y = sess.run(out,feed_dict={_X:test_x})
    mse = tf.reduce_mean(tf.square(pred_y - test_y.values))
    mse_ = sess.run(mse)
    accuracy = (sess.run(accuracy,feed_dict={_X:train_x, _Y:train_y}))
    accuracy_history.append(accuracy)
    print('epoch: ', epoch,' - ', 'cost: ', cost, " - MSE: ", mse_, "- Train Accuracy: ", accuracy)
    